In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

## Feature Engineering pt. 1

In [2]:
dataset = pd.read_csv('../0_data/credit_simple.csv', sep=';')
dataset.shape

(1000, 8)

In [3]:
dataset.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,CLASSE
0,1169.0,4,67,nenhum,01/01/2019,masculino solteiro,radio/tv,bom
1,5951.0,2,22,nenhum,01/01/2020,fem div/cas,radio/tv,ruim
2,2096.0,3,49,nenhum,02/01/2020,masculino solteiro,educação,bom
3,7882.0,4,45,nenhum,02/01/2019,masculino solteiro,mobilia/equipamento,bom
4,4870.0,4,53,nenhum,03/01/2018,masculino solteiro,carro novo,ruim


In [4]:
y = dataset['CLASSE']
X = dataset.drop('CLASSE', axis=1)

In [5]:
X.isnull().sum()

SALDO_ATUAL         7
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64

In [6]:
median = X['SALDO_ATUAL'].median()
median

np.float64(2323.0)

In [7]:
X['SALDO_ATUAL'] = X['SALDO_ATUAL'].fillna(median)
X.isnull().sum()

SALDO_ATUAL         0
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64

In [8]:
agg = X.groupby('ESTADOCIVIL').size()
agg

ESTADOCIVIL
fem div/cas               308
masculino casado/viuvo     92
masculino div/sep          50
masculino solteiro        542
dtype: int64

In [9]:
X['ESTADOCIVIL'] = X['ESTADOCIVIL'].fillna('masculino solteiro')
X.isnull().sum()

SALDO_ATUAL         0
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         0
PROPOSITO           0
dtype: int64

In [10]:
stddev = X['SALDO_ATUAL'].std()
stddev

np.float64(685936688.9820067)

In [11]:
X.loc[X['SALDO_ATUAL'] >= 2 * stddev, 'SALDO_ATUAL'] = median

## Feature Engineering pt. 2

In [12]:
agg = X.groupby(['PROPOSITO']).size()
agg

PROPOSITO
Eletrodomésticos        12
carro novo             234
carro usado            103
educação                50
mobilia/equipamento    181
negócios                97
obras                   22
outros                  12
qualificação             9
radio/tv               280
dtype: int64

In [13]:
X.loc[X['PROPOSITO'].isin(['Eletrodomésticos','qualificação']), 'PROPOSITO'] = 'outros'

In [14]:
agg = X.groupby(['PROPOSITO']).size()
agg

PROPOSITO
carro novo             234
carro usado            103
educação                50
mobilia/equipamento    181
negócios                97
obras                   22
outros                  33
radio/tv               280
dtype: int64

In [15]:
X['DATA']

0      01/01/2019
1      01/01/2020
2      02/01/2020
3      02/01/2019
4      03/01/2018
          ...    
995    29/06/2018
996    30/06/2018
997    03/07/2018
998    04/07/2019
999    05/07/2018
Name: DATA, Length: 1000, dtype: object

In [16]:
X['DATA'] = pd.to_datetime(X['DATA'], format='%d/%m/%Y')
X['ANO'] = X['DATA'].dt.year
X['MES'] = X['DATA'].dt.month
X['DIASEMANA'] = X['DATA'].dt.day_name()
X

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA
0,1169.0,4,67,nenhum,2019-01-01,masculino solteiro,radio/tv,2019,1,Tuesday
1,5951.0,2,22,nenhum,2020-01-01,fem div/cas,radio/tv,2020,1,Wednesday
2,2096.0,3,49,nenhum,2020-01-02,masculino solteiro,educação,2020,1,Thursday
3,7882.0,4,45,nenhum,2019-01-02,masculino solteiro,mobilia/equipamento,2019,1,Wednesday
4,4870.0,4,53,nenhum,2018-01-03,masculino solteiro,carro novo,2018,1,Wednesday
...,...,...,...,...,...,...,...,...,...,...
995,1736.0,4,31,nenhum,2018-06-29,fem div/cas,mobilia/equipamento,2018,6,Friday
996,3857.0,4,40,nenhum,2018-06-30,masculino div/sep,carro usado,2018,6,Saturday
997,804.0,4,38,nenhum,2018-07-03,masculino solteiro,radio/tv,2018,7,Tuesday
998,1845.0,4,23,nenhum,2019-07-04,masculino solteiro,radio/tv,2019,7,Thursday


In [17]:
print(X['ESTADOCIVIL'].unique())
print(X['PROPOSITO'].unique())
print(X['DIASEMANA'].unique())
print(X['OUTROSPLANOSPGTO'].unique())

['masculino solteiro' 'fem div/cas' 'masculino div/sep'
 'masculino casado/viuvo']
['radio/tv' 'educação' 'mobilia/equipamento' 'carro novo' 'carro usado'
 'negócios' 'outros' 'obras']
['Tuesday' 'Wednesday' 'Thursday' 'Saturday' 'Sunday' 'Monday' 'Friday']
['nenhum' 'banco' 'stores']


In [18]:
labelencoder = LabelEncoder()
X['ESTADOCIVIL'] = labelencoder.fit_transform(X['ESTADOCIVIL'])
X['PROPOSITO'] = labelencoder.fit_transform(X['PROPOSITO'])
X['DIASEMANA'] = labelencoder.fit_transform(X['DIASEMANA'])
X

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA
0,1169.0,4,67,nenhum,2019-01-01,3,7,2019,1,5
1,5951.0,2,22,nenhum,2020-01-01,0,7,2020,1,6
2,2096.0,3,49,nenhum,2020-01-02,3,2,2020,1,4
3,7882.0,4,45,nenhum,2019-01-02,3,3,2019,1,6
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,6
...,...,...,...,...,...,...,...,...,...,...
995,1736.0,4,31,nenhum,2018-06-29,0,3,2018,6,0
996,3857.0,4,40,nenhum,2018-06-30,2,1,2018,6,2
997,804.0,4,38,nenhum,2018-07-03,3,7,2018,7,5
998,1845.0,4,23,nenhum,2019-07-04,3,7,2019,7,4


In [19]:
X = pd.get_dummies(X, columns=['OUTROSPLANOSPGTO'], prefix='OUTROS')
X

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA,OUTROS_banco,OUTROS_nenhum,OUTROS_stores
0,1169.0,4,67,2019-01-01,3,7,2019,1,5,False,True,False
1,5951.0,2,22,2020-01-01,0,7,2020,1,6,False,True,False
2,2096.0,3,49,2020-01-02,3,2,2020,1,4,False,True,False
3,7882.0,4,45,2019-01-02,3,3,2019,1,6,False,True,False
4,4870.0,4,53,2018-01-03,3,0,2018,1,6,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
995,1736.0,4,31,2018-06-29,0,3,2018,6,0,False,True,False
996,3857.0,4,40,2018-06-30,2,1,2018,6,2,False,True,False
997,804.0,4,38,2018-07-03,3,7,2018,7,5,False,True,False
998,1845.0,4,23,2019-07-04,3,7,2019,7,4,False,True,False


In [20]:
# SALDO_ATUAL, RESIDENCIADESDE, IDADE
sc = StandardScaler()
m = sc.fit_transform(X.iloc[:, 0:3])

In [21]:
X = pd.concat([X, pd.DataFrame(m, columns=['SALDO_ATUAL_sc', 'RESIDENCIADESDE_sc', 'IDADE_sc'])], axis=1)
X = X.drop(['SALDO_ATUAL', 'RESIDENCIADESDE', 'IDADE', 'DATA', 'OUTROS_banco'], axis=1)
X

,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA,OUTROS_nenhum,OUTROS_stores,SALDO_ATUAL_sc,RESIDENCIADESDE_sc,IDADE_sc
0,3,7,2019,1,5,True,False,-0.745516,1.046987,1.639276
1,0,7,2020,1,6,True,False,0.957740,-0.765977,-0.740241
2,3,2,2020,1,4,True,False,-0.415337,0.140505,0.687469
3,3,3,2019,1,6,True,False,1.645526,1.046987,0.475956
4,3,0,2018,1,6,True,False,0.572709,1.046987,0.898982
...,...,...,...,...,...,...,...,...,...,...
995,0,3,2018,6,0,True,False,-0.543562,1.046987,-0.264338
996,2,1,2018,6,2,True,False,0.211898,1.046987,0.211566
997,3,7,2018,7,5,True,False,-0.875522,1.046987,0.105809
998,3,7,2019,7,4,True,False,-0.504738,1.046987,-0.687363
